In [ ]:
import csv
from transformers import pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [2]:
data = "../Files/imdb_master.csv"
aux = {"neg": 0, "pos": 1}
texts = []
labels = []

with open(data, encoding="ISO-8859-1") as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        if not (row['label'] == "unsup"):
            texts.append(row['review'])
            labels.append(aux[row['label']])

texts = texts[int(len(texts) / 2) :]
labels = labels[int(len(labels) / 2) :]

In [3]:
def yield_data(texts):
    for text in texts:
        yield text

In [4]:
model_paths = ['siebert/sentiment-roberta-large-english',
               'bhadresh-savani/distilbert-base-uncased-sentiment-sst2',
               'cardiffnlp/twitter-roberta-base-sentiment']

In [5]:
model_preds = {}

for model_path in model_paths:
    model = pipeline("sentiment-analysis", model=model_path, tokenizer=model_path, max_length=512, truncation=True, batch_size=500)

    preds = []
    for pred in model(yield_data(texts)):
        preds.append(pred)
    
    model_preds[model_path] = preds

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at siebert/sentiment-roberta-large-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
Some layers from the model checkpoint at bhadresh-savani/distilbert-base-uncased-sentiment-sst2 were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identi

In [6]:
model_preds.keys()

dict_keys(['siebert/sentiment-roberta-large-english', 'bhadresh-savani/distilbert-base-uncased-sentiment-sst2', 'cardiffnlp/twitter-roberta-base-sentiment'])

In [8]:
label_aux = {'NEGATIVE': 0, 'POSITIVE': 1, 'NEUTRAL': 1}

model_pred = model_preds['siebert/sentiment-roberta-large-english']
processed_pred = [label_aux[pred['label']] for pred in model_pred];

print(confusion_matrix(labels, processed_pred))
print(classification_report(labels, processed_pred))

[[11820   680]
 [  603 11897]]
              precision    recall  f1-score   support

           0       0.95      0.95      0.95     12500
           1       0.95      0.95      0.95     12500

    accuracy                           0.95     25000
   macro avg       0.95      0.95      0.95     25000
weighted avg       0.95      0.95      0.95     25000



In [9]:
label_aux = {'NEGATIVE': 0, 'POSITIVE': 1}

model_pred = model_preds['bhadresh-savani/distilbert-base-uncased-sentiment-sst2']
processed_pred = [label_aux[pred['label']] for pred in model_pred];

print(confusion_matrix(labels, processed_pred))
print(classification_report(labels, processed_pred))

[[11387  1113]
 [ 1953 10547]]
              precision    recall  f1-score   support

           0       0.85      0.91      0.88     12500
           1       0.90      0.84      0.87     12500

    accuracy                           0.88     25000
   macro avg       0.88      0.88      0.88     25000
weighted avg       0.88      0.88      0.88     25000



In [10]:
label_aux = {'LABEL_0': 0, 'LABEL_1': 1, 'LABEL_2': 1}

model_pred = model_preds['cardiffnlp/twitter-roberta-base-sentiment']
processed_pred = [label_aux[pred['label']] for pred in model_pred];

print(confusion_matrix(labels, processed_pred))
print(classification_report(labels, processed_pred))

[[ 9884  2616]
 [  913 11587]]
              precision    recall  f1-score   support

           0       0.92      0.79      0.85     12500
           1       0.82      0.93      0.87     12500

    accuracy                           0.86     25000
   macro avg       0.87      0.86      0.86     25000
weighted avg       0.87      0.86      0.86     25000



In [18]:
label_aux = {'NEGATIVE': 0, 'POSITIVE': 1, 'NEUTRAL': 1}
label_aux2 = {'LABEL_0': 0, 'LABEL_1': 1, 'LABEL_2': 1}

processed_preds = {}

model_pred = model_preds['siebert/sentiment-roberta-large-english']
processed_preds['siebert/sentiment-roberta-large-english'] = [(label_aux[pred['label']], pred['score']) for pred in model_pred];

model_pred = model_preds['bhadresh-savani/distilbert-base-uncased-sentiment-sst2']
processed_preds['bhadresh-savani/distilbert-base-uncased-sentiment-sst2'] = [(label_aux[pred['label']], pred['score']) for pred in model_pred];

model_pred = model_preds['cardiffnlp/twitter-roberta-base-sentiment']
processed_preds['cardiffnlp/twitter-roberta-base-sentiment'] = [(label_aux2[pred['label']], pred['score']) for pred in model_pred];

In [22]:
# Metodo da soma

final_result = []
for index in range(len(labels)):
    positive_porcentage = 0
    negative_porcentage = 0
    for pred in processed_preds:
        if processed_preds[pred][index][0] == 1:
            positive_porcentage += processed_preds[pred][index][1]
            negative_porcentage += 1 - processed_preds[pred][index][1]
        else:
            positive_porcentage += 1 - processed_preds[pred][index][1]
            negative_porcentage += processed_preds[pred][index][1]

    if positive_porcentage > negative_porcentage:
        final_result.append(1)
    else:
        final_result.append(0)

print(confusion_matrix(labels, final_result))
print(classification_report(labels, final_result))

[[11768   732]
 [  896 11604]]
              precision    recall  f1-score   support

           0       0.93      0.94      0.94     12500
           1       0.94      0.93      0.93     12500

    accuracy                           0.93     25000
   macro avg       0.93      0.93      0.93     25000
weighted avg       0.93      0.93      0.93     25000



In [27]:
# Metodo do produto

final_result = []
for index in range(len(labels)):
    positive_porcentage = 1
    negative_porcentage = 1
    for pred in processed_preds:
        if processed_preds[pred][index][0] == 1:
            positive_porcentage = positive_porcentage * processed_preds[pred][index][1]
            negative_porcentage = negative_porcentage * (1 - processed_preds[pred][index][1])
        else:
            positive_porcentage = positive_porcentage * (1 - processed_preds[pred][index][1])
            negative_porcentage = negative_porcentage * processed_preds[pred][index][1]

    if positive_porcentage > negative_porcentage:
        final_result.append(1)
    else:
        final_result.append(0)

print(confusion_matrix(labels, final_result))
print(classification_report(labels, final_result))

[[11835   665]
 [  650 11850]]
              precision    recall  f1-score   support

           0       0.95      0.95      0.95     12500
           1       0.95      0.95      0.95     12500

    accuracy                           0.95     25000
   macro avg       0.95      0.95      0.95     25000
weighted avg       0.95      0.95      0.95     25000



In [28]:
# Metodo da media

final_result = []
for index in range(len(labels)):
    positive_porcentage = 0
    negative_porcentage = 0
    for pred in processed_preds:
        if processed_preds[pred][index][0] == 1:
            positive_porcentage += processed_preds[pred][index][1]
            negative_porcentage += (1 - processed_preds[pred][index][1])
        else:
            positive_porcentage += (1 - processed_preds[pred][index][1])
            negative_porcentage += processed_preds[pred][index][1]

    if (positive_porcentage / 3) > (negative_porcentage / 3):
        final_result.append(1)
    else:
        final_result.append(0)

print(confusion_matrix(labels, final_result))
print(classification_report(labels, final_result))

[[11768   732]
 [  896 11604]]
              precision    recall  f1-score   support

           0       0.93      0.94      0.94     12500
           1       0.94      0.93      0.93     12500

    accuracy                           0.93     25000
   macro avg       0.93      0.93      0.93     25000
weighted avg       0.93      0.93      0.93     25000



In [29]:
# regra do minimo

final_result = []
for index in range(len(labels)):
    positive_porcentage = []
    negative_porcentage = []
    for pred in processed_preds:
        if processed_preds[pred][index][0] == 1:
            positive_porcentage.append(processed_preds[pred][index][1])
            negative_porcentage.append((1 - processed_preds[pred][index][1]))
        else:
            positive_porcentage.append((1 - processed_preds[pred][index][1]))
            negative_porcentage.append(processed_preds[pred][index][1])

    if min(positive_porcentage) > min(negative_porcentage):
        final_result.append(1)
    else:
        final_result.append(0)

print(confusion_matrix(labels, final_result))
print(classification_report(labels, final_result))

[[11842   658]
 [  636 11864]]
              precision    recall  f1-score   support

           0       0.95      0.95      0.95     12500
           1       0.95      0.95      0.95     12500

    accuracy                           0.95     25000
   macro avg       0.95      0.95      0.95     25000
weighted avg       0.95      0.95      0.95     25000



In [30]:
# Regra do maximo

final_result = []
for index in range(len(labels)):
    positive_porcentage = []
    negative_porcentage = []
    for pred in processed_preds:
        if processed_preds[pred][index][0] == 1:
            positive_porcentage.append(processed_preds[pred][index][1])
            negative_porcentage.append((1 - processed_preds[pred][index][1]))
        else:
            positive_porcentage.append((1 - processed_preds[pred][index][1]))
            negative_porcentage.append(processed_preds[pred][index][1])

    if max(positive_porcentage) > max(negative_porcentage):
        final_result.append(1)
    else:
        final_result.append(0)

print(confusion_matrix(labels, final_result))
print(classification_report(labels, final_result))

[[11842   658]
 [  636 11864]]
              precision    recall  f1-score   support

           0       0.95      0.95      0.95     12500
           1       0.95      0.95      0.95     12500

    accuracy                           0.95     25000
   macro avg       0.95      0.95      0.95     25000
weighted avg       0.95      0.95      0.95     25000



In [31]:
# Usando voto majoritario

final_result = []
for index in range(len(labels)):
    positive_votes = 0
    negative_votes = 0
    for pred in processed_preds:
        if processed_preds[pred][index][0] == 1:
            positive_votes += 1
        else:
            negative_votes += 1

    if positive_votes > negative_votes:
        final_result.append(1)
    else:
        final_result.append(0)

print(confusion_matrix(labels, final_result))
print(classification_report(labels, final_result))

[[11431  1069]
 [  801 11699]]
              precision    recall  f1-score   support

           0       0.93      0.91      0.92     12500
           1       0.92      0.94      0.93     12500

    accuracy                           0.93     25000
   macro avg       0.93      0.93      0.93     25000
weighted avg       0.93      0.93      0.93     25000

